# RGI04 (Arctic Canada South)

F. Maussion

Full replace by Philipp and Frank, except Labrador which still used RGI6

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist, find_duplicates
import os

## Files and storage paths

In [ ]:
# Region of interest
reg = 4

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# RGI v6 file for comparison later 
rgi6_reg_file = os.path.join(data_dir, 'l0_RGIv6', '04_rgi60_ArcticCanadaNorth.zip')

### Load the input data

In [ ]:
# Read L2 files
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

### List of submissions 

In [ ]:
sdf, _ = submission_summary(shp)
sdf

Notes based on individual submission evaluations: 

- 589: This is the RGI6
- 745: also RGI6, corrected: for https://github.com/GLIMS-RGI/rgi7_scripts/issues/8
- 750: this is the new submission by F&P - valid above 61°N, below we need RGI6: https://github.com/GLIMS-RGI/rgi7_scripts/issues/57

In [ ]:
# # Write out selection in intermediate shape files for manual GIS review
# tmp_output_dir = mkdir(os.path.join(data_dir, 'l0_tmp_data', f'rgi{reg:02d}_inventories'))
# tmp_output_dir_tar = mkdir(os.path.join(data_dir, 'l0_tmp_data'))
# for subid in shp.subm_id.unique():
#     s_loc = shp.loc[shp.subm_id == subid]
#     s_loc.to_file(tmp_output_dir + f'/subm_{int(subid):03d}.shp')
# print('Taring...')
# print(subprocess.run(['tar', '-zcvf', f'{tmp_output_dir_tar}/rgi{reg:02d}_inventories.tar.gz', '-C', 
#                       os.path.join(data_dir, 'l0_tmp_data'), f'rgi{reg:02d}_inventories']))

### Apply selection criteria to create the RGI7 data subset

In [ ]:
rgi7_north = shp.loc[shp['subm_id'].isin([750])].copy()
print(needs_size_filter(rgi7_north))
rgi7_north['is_rgi6'] = False

In [ ]:
rgi7_south = shp.loc[shp['subm_id'].isin([589, 745])].copy()
rgi7_south = rgi7_south.loc[rgi7_south.CenLat < 61]
print(needs_size_filter(rgi7_south))
rgi7_south['is_rgi6'] = True

In [ ]:
# combine the geodataframes
rgi7 = pd.concat([rgi7_north, rgi7_south])

### Some sanity checks 

In [ ]:
dupes = find_duplicates(rgi7)

In [ ]:
sdf, df_class = submission_summary(rgi7)
df_class

In [ ]:
# Check the orphaned rock outcrops
orphan_f = os.path.join(data_dir, 'l1_orphan_interiors', f'RGI{reg:02d}', f'RGI{reg:02d}.shp')
if os.path.exists(orphan_f):
    orphan_f = gpd.read_file(orphan_f)
    if np.any(np.isin(rgi7.subm_id.unique(), orphan_f.subm_id.unique())):
        print('Orphan rock outcrops detected.')

### Plots 

In [ ]:
plot_map(rgi7, reg)

In [ ]:
plot_map(rgi7, reg, is_rgi6=True)

In [ ]:
plot_date_hist(rgi7, reg)

### Text for github

In [ ]:
fgh = sdf.T
fgh

In [ ]:
print(fgh.to_markdown(headers=np.append(['subm_id'], fgh.columns)))

## Write out and tar 

In [ ]:
dd = mkdir(f'{output_dir}/RGI{reg:02d}/', reset=True)

print('Writing...')
rgi7.to_file(dd + f'RGI{reg:02d}.shp')

print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{output_dir_tar}/RGI{reg:02d}.tar.gz', '-C', output_dir, f'RGI{reg:02d}']))

## Consistency checks

### With RGI6 (South) 

In [ ]:
from utils import open_zip_shapefile
rgi6 = open_zip_shapefile(rgi6_reg_file)

In [ ]:
rgi6 = rgi6.loc[rgi6.CenLat < 61]

In [ ]:
len(rgi7_south), len(rgi6)

Test the areas:

In [ ]:
rgi6['Area_us'] = rgi6.to_crs({'proj':'cea'}).area 
rgi7_south['area'] = rgi7.to_crs({'proj':'cea'}).area 

In [ ]:
print('Area RGI7a (km2)', rgi7_south['area'].sum() * 1e-6)
print('Area RGI6 (km2)', rgi6['Area'].sum())
print('Area RGI6 computed by us (km2)', rgi6['Area_us'].sum() * 1e-6)
print('diff areas RGI6 - RGI7 computed by us (km2)', (rgi6['Area_us'].sum() - rgi7_south['area'].sum()) * 1e-6)

### With F&P submission 

In [ ]:
ref_pr = open_zip_shapefile(data_dir + '/l0_support_data/c3s_gi_rgi04_L7_2000_v1.zip')
ref_pr['area'] = ref_pr.to_crs({'proj':'cea'}).area 

In [ ]:
print('Number of glaciers in RGI7 subset:', len(rgi7_north))
print('Number of glaciers in reference data (FP):', len(ref_pr))
print('Difference:', len(rgi7_north)-len(ref_pr))

In [ ]:
print('Area RGI7a (km2)', rgi7_north['area'].sum() * 1e-6)
print('Area Ref (km2)', ref_pr['area'].sum())
print('diff areas Ref - RGI7 (km2)', (ref_pr['area'].sum() - rgi7_north['area'].sum()) * 1e-6)

### With RGI6 (Full: differences do not depict problems, just differences) 

In [ ]:
rgi6 = open_zip_shapefile(rgi6_reg_file)

In [ ]:
len(rgi7), len(rgi6)

Test the areas:

In [ ]:
rgi6['Area_us'] = rgi6.to_crs({'proj':'cea'}).area 
rgi7['area'] = rgi7.to_crs({'proj':'cea'}).area 

In [ ]:
print('Area RGI7a (km2)', rgi7['area'].sum() * 1e-6)
print('Area RGI6 (km2)', rgi6['Area'].sum())
print('Area RGI6 computed by us (km2)', rgi6['Area_us'].sum() * 1e-6)
print('diff areas RGI6 - RGI7 computed by us (km2)', (rgi6['Area_us'].sum() - rgi7['area'].sum()) * 1e-6)